In [2]:
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import os
import sys
import numpy as np
import cmocean.cm as ocm
import gsw
from unesco import unesco

proj_dir = os.path.join(os.pardir,os.pardir)
data_dir = os.path.join(proj_dir,'data','analysis')
int_dir = os.path.join(proj_dir,'data','analysis','interim')
src_dir = os.path.join(proj_dir,'src')
fig_dir = os.path.join(proj_dir,'reports','devel','figures')

sys.path.append(src_dir)
%load_ext autoreload
%autoreload 1

tools_dir = os.path.join(proj_dir,'src','tools')
sys.path.append(tools_dir)

#from tools.roms_ds_tools import make_depth_single_time,make_cartesian_grid_3D_single_time
#%aimport tools.roms_ds_tools

#from tools.log_progress import log_progress

#from visualization.shiftedColorMap import shiftedColorMap
#from tools.calc_z import calc_z
#from tools.make_masks import make_mask_sector,make_mask_shelf_sector,make_mask_ice_shelves
#import tools.make_sose_ds as sose_ds
#from tools.regrid import regrid

In [3]:
# Number of temperature and salinity bins
num_bins = 1000
# Bounds on temperature and salinity bins (pre-computed, change if needed)
min_salt = 32.1
max_salt = 35.2
min_temp = -3.5
max_temp = 4

temp_bins = np.linspace(min_temp, max_temp, num=num_bins)
# Calculate centres of temperature bins (for plotting)
temp_centres = 0.5*(temp_bins[:-1] + temp_bins[1:])
# Repeat for salinity
salt_bins = np.linspace(min_salt, max_salt, num=num_bins)
salt_centres = 0.5*(salt_bins[:-1] + salt_bins[1:])

freezing_pt_roms = salt_centres/(-18.48 + 18.48/1e3*salt_centres)
salt_2d, temp_2d = np.meshgrid(salt_centres, temp_centres)

density = unesco(temp_2d, salt_2d, np.zeros(np.shape(temp_centres)))-1000
# Density contours to plot
density_lev = np.arange(26.6, 28.4, 0.2)

In [24]:
woa_path = os.path.join(data_dir,'external','woa','woa18_A5B7_t15_01.nc')
woa_t = xr.open_dataset(woa_path,decode_times=False).squeeze()
woa_path = os.path.join(data_dir,'external','woa','woa18_A5B7_s15_01.nc')
woa_s = xr.open_dataset(woa_path,decode_times=False).squeeze()

In [25]:
woa_s.s_an

<xarray.DataArray 's_an' (depth: 102, lat: 26, lon: 360)>
[954720 values with dtype=float32]
Coordinates:
    time     float32 79.5
  * depth    (depth) float32 0.0 5.0 10.0 15.0 ... 5200.0 5300.0 5400.0 5500.0
  * lat      (lat) float32 -89.5 -88.5 -87.5 -86.5 ... -67.5 -66.5 -65.5 -64.5
  * lon      (lon) float32 -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
Attributes:
    standard_name:  sea_water_salinity
    long_name:      Objectively analyzed mean fields for sea_water_salinity a...
    cell_methods:   area: mean depth: mean time: mean within years time: mean...
    grid_mapping:   crs
    units:          1e-3

In [59]:
p-depth_2d

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [5.43298235e-02, 5.43135150e-02, 5.42809182e-02, ...,
        5.00774549e-02, 4.97306839e-02, 4.93732683e-02],
       [1.08783690e-01, 1.08751072e-01, 1.08685877e-01, ...,
        1.00278749e-01, 9.95851900e-02, 9.88703419e-02],
       ...,
       [1.25155608e+02, 1.25137902e+02, 1.25102513e+02, ...,
        1.20538962e+02, 1.20162489e+02, 1.19774460e+02],
       [1.28776410e+02, 1.28758363e+02, 1.28722290e+02, ...,
        1.24070662e+02, 1.23686923e+02, 1.23291405e+02],
       [1.32442313e+02, 1.32423924e+02, 1.32387168e+02, ...,
        1.27647393e+02, 1.27256382e+02, 1.26853369e+02]])

In [79]:
t = woa_t.t_an
SP = woa_s.s_an

lon_2d,depth_2d = np.meshgrid(t.lon,t.depth)
lat_2d,depth_2d = np.meshgrid(t.lat,t.depth)
p = gsw.p_from_z(-depth_2d,lat_2d)
p_3d = np.repeat(p[:,:,np.newaxis],t.lon.size,axis=2)
lon_3d = np.repeat(lon_2d[:,np.newaxis,:],t.lat.size,axis=1)
lat_3d = np.repeat(lat_2d[:,:,np.newaxis],t.lon.size,axis=2)
SA = gsw.SA_from_SP(SP,p_3d,lon_3d,lat_3d)
pt = gsw.pt0_from_t(SA,t,p_3d)

woa_t['pt']=xr.DataArray(pt,dims=('depth','lat','lon'))

In [81]:
temp.shape

(102, 26, 360)

In [85]:

depth_3d.shape

(102, 26, 360)

In [86]:
lon_2d,lat_2d = np.meshgrid(woa_t.lon,woa_t.lat)
#mask = np.zeros_like(lat_2d,dtype=int)
#mask[lat_2d < -65] = 1

lat_min = lat_2d.min()
lat_max = lat_2d.max()
lon_min = lon_2d.min()
lon_max = lon_2d.max()

temp = woa_t.pt
salt = woa_s.s_an
depth_3d = np.repeat(depth_2d[:,:,np.newaxis],t.lon.size,axis=2)

In [87]:
def calc_dx_dy(longitude,latitude,shape,radius=6370997.):
    ''' This definition calculates the distance 
        between grid points that are in
        a latitude/longitude format.
        
        Using pyproj GEOD; different Earth Shapes 
        https://jswhit.github.io/pyproj/pyproj.Geod-class.html
        Common shapes: 'sphere', 'WGS84', 'GRS80'
        
        Accepts, 1D arrays for latitude and longitude
        
        Returns: dx, dy; 2D arrays of distances 
                       between grid points in the x and y direction in meters 
    '''
    from pyproj import Geod
    
    if (radius != 6370997.):
        g = Geod(a=radius,b=radius)
    else:
        g = Geod(ellps=shape)
    
    dx = np.empty(latitude.shape)
    dy = np.zeros(longitude.shape)
    
    for i in range(latitude.shape[1]):
        for j in range(latitude.shape[0]-1):
            _, _, dx[j,i] = g.inv(longitude[j,i],latitude[j,i],longitude[j+1,i],latitude[j+1,i])
    dx[j+1,:] = dx[j,:]
    
    for i in range(latitude.shape[1]-1):
        for j in range(latitude.shape[0]):
            _, _, dy[j,i] = g.inv(longitude[j,i],latitude[j,i],longitude[j,i+1],latitude[j,i+1])
    dy[:,i+1] = dy[:,i]
    
    return dx, dy

In [134]:
mask = ~woa_s.s_an.isnull()

In [125]:
dx,dy = calc_dx_dy(lon_2d,lat_2d,'WGS84')

d = t.depth.values
d_l = ((d[1:]-d[:-1])/2)+d[:-1]
dz = np.zeros_like(t.depth)
dz[0] = d_l[0]-d[0]
dz[1:-1] = d_l[1:]-d_l[:-1]
dz[-1] = (d[-1]-d_l[-1])*2

dz_2d = np.repeat(dz[:,np.newaxis],t.lat.size,axis=1)
dz_3d = np.repeat(dz_2d[:,:,np.newaxis],t.lon.size,axis=2)

dV = dx*dy*dz_3d

# Set up 2D arrays of temperature bins x salinity bins to hold average
# depth of water masses, weighted by volume
woa_ts = np.zeros([np.size(temp_centres), np.size(salt_centres)])
# Also arrays to integrate volume
volume = np.zeros([np.size(temp_centres), np.size(salt_centres)])

# Loop over 2D grid boxes
for j in log_progress(np.arange(lon_2d.shape[0])):
    for i in np.arange(lon_2d.shape[1]):
        # Check for land mask
            for k in np.arange(woa_t.depth.size):
                if mask[k,j,i]:
                    # Figure out which bins this falls into
                    temp_index = np.nonzero(temp_bins > temp[k,j,i])[0][0] - 1
                    salt_index = np.nonzero(salt_bins > salt[k,j,i])[0][0] - 1
                    # Integrate depth*dV in this bin
                    woa_ts[temp_index, salt_index] += -depth[k,j,i]*dV[k,j,i]
                    volume[temp_index, salt_index] += dV[k,j,i]

# Mask bins with zero volume
sose_ts_vals = np.ma.masked_where(volume ==0, sose_ts_vals)
volume = np.ma.masked_where(volume ==0, volume)
# Convert depths from integrals to volume-averages
sose_ts_vals /= volume

In [ ]:
lon_2d,lat_2d = np.meshgrid(sds.longitude.values,sds.latitude.values)
mask = np.zeros_like(lat_2d,dtype=int)
mask[lat_2d < -65] = 1

lat_min = lat_2d[mask].min()
lat_max = lat_2d[mask].max()
lon_min = lon_2d[mask].min()
lon_max = lon_2d[mask].max()

temp = sds.temperature.values
salt = sds.salinity.values
depth = sds.depth.values
depth = np.rollaxis(np.tile(depth,(320,2160,1)),-1)
dV = sds.DRC*sds.DXC*sds.DYC

bound_mask = (temp < max_temp) & (temp > min_temp) & \
             (salt > min_salt) & (salt < max_salt)

# Set up 2D arrays of temperature bins x salinity bins to hold average
# depth of water masses, weighted by volume
sose_ts_vals = np.zeros([np.size(temp_centres), np.size(salt_centres)])
# Also arrays to integrate volume
volume = np.zeros([np.size(temp_centres), np.size(salt_centres)])

# Loop over 2D grid boxes
for j in log_progress(np.arange(lon_2d.shape[0])):
    for i in np.arange(lon_2d.shape[1]):
        # Check for land mask
        if mask[j,i]:
            for k in np.arange(sds.depth.size):
                if bound_mask[k,j,i]:
                    # Figure out which bins this falls into
                    temp_index = np.nonzero(temp_bins > temp[k,j,i])[0][0] - 1
                    salt_index = np.nonzero(salt_bins > salt[k,j,i])[0][0] - 1
                    # Integrate depth*dV in this bin
                    sose_ts_vals[temp_index, salt_index] += -depth[k,j,i]*dV[k,j,i]
                    volume[temp_index, salt_index] += dV[k,j,i]

# Mask bins with zero volume
sose_ts_vals = np.ma.masked_where(volume ==0, sose_ts_vals)
volume = np.ma.masked_where(volume ==0, volume)
# Convert depths from integrals to volume-averages
sose_ts_vals /= volume